In [3]:
cd ..

/kaggle


In [4]:
cd input

/kaggle/input


In [5]:
cd tabular-playground-series-dec-2021/

/kaggle/input/tabular-playground-series-dec-2021


In [6]:
ls

sample_submission.csv  test.csv  train.csv


In [7]:
import pandas as pd

In [8]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [9]:
train_df.head(5)

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,0,3189,40,8,30,13,3270,206,234,193,...,0,0,0,0,0,0,0,0,0,1
1,1,3026,182,5,280,29,3270,233,240,106,...,0,0,0,0,0,0,0,0,0,2
2,2,3106,13,7,351,37,2914,208,234,137,...,0,0,0,0,0,0,0,0,0,1
3,3,3022,276,13,192,16,3034,207,238,156,...,0,0,0,0,0,0,0,0,0,2
4,4,2906,186,13,266,22,2916,231,231,154,...,0,0,0,0,0,0,0,0,0,2


In [10]:
x = train_df.drop('Id',axis=1)
test = test_df
submission = test_df['Id']

In [11]:
y = x['Cover_Type']

In [12]:
submission.head(5)

0    4000000
1    4000001
2    4000002
3    4000003
4    4000004
Name: Id, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
import numpy as np 

In [15]:
import xgboost as xgb

xgb_cl = xgb.XGBClassifier()

In [ ]:
model1v = XGBRegressor(max_depth=6,
                       n_estimators=250,
                       learning_rate=0.08,
                       subsample=0.7,
                       alpha=0.5,
                       max_leaves: 255,
                       lambda: 100,
                       random_state=123)                           
        
model1v.fit(x_train, y_train, verbose=100)
oof_pred1 = model1v.predict(x_train)
oof_pred1 = np.clip(oof_pred1, 0.30, y.max())
print(mean_absolute_error(y_train, oof_pred1))

In [16]:
import tensorflow as tf
from glob import glob  
from tensorflow import keras
from tensorflow.keras.models import Sequential
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import datasets, layers, models
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import optimizers
from keras.layers import Dropout

In [ ]:
ann = models.Sequential([
    layers.Dense(54, activation='relu',input_dim=54),
#     layers.Dense(30, activation='relu'), 
#     layers.Dense(15, activation='relu'), 
    layers.Dense(7, activation='relu'), 
    layers.Dense(1, activation='sigmoid')
])

opt = keras.optimizers.Adam(learning_rate=0.001)
ann.compile(optimizer=opt, 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
ann.summary()

In [17]:
train_df["Aspect"][train_df["Aspect"] < 0] += 360
train_df["Aspect"][train_df["Aspect"] > 359] -= 360

test_df["Aspect"][test_df["Aspect"] < 0] += 360
test_df["Aspect"][test_df["Aspect"] > 359] -= 360
train_df.loc[train_df["Hillshade_9am"] < 0, "Hillshade_9am"] = 0
test_df.loc[test_df["Hillshade_9am"] < 0, "Hillshade_9am"] = 0

train_df.loc[train_df["Hillshade_Noon"] < 0, "Hillshade_Noon"] = 0
test_df.loc[test_df["Hillshade_Noon"] < 0, "Hillshade_Noon"] = 0

train_df.loc[train_df["Hillshade_3pm"] < 0, "Hillshade_3pm"] = 0
test_df.loc[test_df["Hillshade_3pm"] < 0, "Hillshade_3pm"] = 0

train_df.loc[train_df["Hillshade_9am"] > 255, "Hillshade_9am"] = 255
test_df.loc[test_df["Hillshade_9am"] > 255, "Hillshade_9am"] = 255

train_df.loc[train_df["Hillshade_Noon"] > 255, "Hillshade_Noon"] = 255
test_df.loc[test_df["Hillshade_Noon"] > 255, "Hillshade_Noon"] = 255

train_df.loc[train_df["Hillshade_3pm"] > 255, "Hillshade_3pm"] = 255
test_df.loc[test_df["Hillshade_3pm"] > 255, "Hillshade_3pm"] = 255

In [18]:
x = train_df
x = x.drop('Cover_Type',axis=1)
x = x.drop('Id',axis=1)

In [ ]:
ann.fit(x_train,y_train,epochs=100, batch_size = 1024, validation_data=(x_test, y_test))

In [19]:
from sklearn.metrics import accuracy_score

# Init classifier
xgb_cl = xgb.XGBClassifier()

# Fit
xgb_cl.fit(x, y)

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:41:02] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=2,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [20]:
test = test.drop('Id',axis=1)

In [21]:
test.head(5)

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,2763,78,20,377,88,3104,218,213,195,1931,...,0,0,0,0,0,0,0,0,0,0
1,2826,153,11,264,39,295,219,238,148,2557,...,0,0,0,0,0,0,0,0,0,0
2,2948,57,19,56,44,852,202,217,163,1803,...,0,0,1,0,0,0,0,0,0,0
3,2926,119,6,158,134,2136,234,240,142,857,...,0,0,0,0,0,0,0,0,0,0
4,2690,10,4,38,108,3589,213,221,229,431,...,0,0,0,0,0,0,0,0,0,0


In [22]:
preds = xgb_cl.predict(test)

In [23]:
preds.shape

(1000000,)

In [24]:
submission[0]

4000000

In [25]:
sample = pd.read_csv('sample_submission.csv')

In [26]:
sample.head(5)

,Id,Cover_Type
0,4000000,2
1,4000001,2
2,4000002,2
3,4000003,2
4,4000004,2


In [27]:
sample['Cover_Type'] = preds

sample['Id'] = test_df['Id']

In [28]:
cd ..

/kaggle/input


In [29]:
cd ..

/kaggle


In [30]:
cd working/

/kaggle/working


In [31]:
sample.to_csv("submission_xgb.csv",index=False)

In [32]:
ls

__notebook_source__.ipynb  submission_xgb.csv


In [35]:
from IPython.display import FileLink
FileLink(r'submission_xgb.csv')

/kaggle/working/submission_xgb.csv